<a href="https://colab.research.google.com/github/JoaoVLima/ActualSimplePDFValidator/blob/main/TDE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUCPR
## **João Victor de Lima**
### TDE01 - Construção de Interpretadores
**Desenvolver um programa que analise documentos no
formato simplepdf**, uma versão textual simplificada inspirada na estrutura interna de arquivos pdf.
O programa deverá **processar documentos simplepdf**, **validar sua estrutura segundo as regras formais da linguagem que aqui será definida**, **gerar estatísticas** e **transformações do conteúdo**.

[adobe pdf docs (for reference)](https://opensource.adobe.com/dc-acrobat-sdk-docs/pdfstandards/pdfreference1.0.pdf)

# Explicação

## Formato SimplePDF

O SimplePDF é combosto por:

1. **Cabeçalho**: Todo documento começa com uma linha de cabeçalho
```
%SPDF-1.0
```

2. **Objetos**: O documento consiste em objetos numerados no formato
```
obj_id obj_gen obj
<< conteudo >>
endobj
```
No qual:
  - **obj_id** é um número inteiro único que identifica o objeto
  - **obj_gen** é um número de geração
(normalmente 0)

3. **Tipos de dados básicos**:
  - **Números**: inteiros (123) ou decimais (123.45)
  - **Strings**: delimitadas por parênteses: (Este é um texto)
  - **Nomes**: começam com /: /Nome
  - **Booleanos**: true ou false
  - **Null**: null
  - **Referências**: indicam referência a outro objeto: obj_id obj_gen R

4. **Estruturas de dados**:
  - **Dicionários**: pares chave/valor no formato: /Chave valor
  - **Arrays**: sequências de valores delimitadas por [ e ]
  - **Streams**: Blocos de dados entre stream e endstream

5. **Estrutura hierárquica**:
  - **Objeto catálogo (root)**: Define a estrutura do documento
  - **Objeto de páginas**: Mantém lista de páginas
  - **Objetos de página**: Contêm conteúdo e propriedades de cada página
  - **Objetos de conteúdo**: Contêm texto e elementos gráficos


6. **Comentários**:
  Linhas que começam com % (exceto o cabeçalho) são comentários

7. **Cross-reference table (xref)**: Tabela que lista a posição de cada objeto no arquivo:
```
xref
0 3
0000000000 65535 f
0000000010 00000 n
0000000079 00000 n
```

  1. ESTRUTURA E PROPÓSITO DA TABELA XREF

    A tabela xref contém a posição (offset em bytes) de cada objeto no arquivo. Sua estrutura consiste em:
    1. A palavra-chave xref em uma linha sozinha
    2. Uma linha com dois números: o primeiro objeto na seção e o número de entradas
    3. Uma entrada para cada objeto, cada uma com exatamente 20 bytes:
    - Offset de 10 dígitos (com zeros à esquerda)
    - Um espaço
    - Número de geração de 5 dígitos (geralmente 00000)
    - Um espaço
    - Um caractere 'n' (para objetos em uso) ou 'f' (para objetos livres)
    - Um espaço e uma quebra de linha
  2. Escreva o conteúdo do documento até antes da seção xref  
  - Inclua o cabeçalho (%SPDF-1.0)
  - Crie todos os objetos numerados
  3. Anote a posição (offset) do início de cada objeto  
  - A posição é o número de bytes desde o início do arquivo
  - Para o objeto 0 (especial), sempre use offset 0, geração 65535 e flag 'f'
  4. Crie a tabela xref usando os offsets anotados  
  Comece com a linha "xref"
  - Na próxima linha, coloque "0 N" onde N é o número total de objetos + 1 (inclui o objeto 0)
  - Para cada objeto, adicione uma entrada de 20 bytes
  5. Complete com a seção trailer  
  - Adicione a palavra-chave "trailer" seguida pelo dicionário trailer
  - Adicione "startxref" seguido pela posição da palavra-chave "xref"
  - Termine com "%%EOF"

8. **Trailer**: Informações finais do documento:
```
trailer
<< /Size 3 /Root 1 0 R >>
startxref
183
%%EOF
```

## Funcionalidades Requeridas do Programa

O programa deve implementar as seguintes funcionalidades:

1. **Validação Estrutural**:
  - Verificar se o cabeçalho SimplePDF está correto
  - Validar a sintaxe de cada objeto e suas propriedades
  - Verificar se todas as referências apontam para objetos existentes
  - Validar a estrutura hierárquica (catálogo → páginas → página)
  - Verificar consistência da tabela xref
  - Validar o trailer

2. **Extração de Informações**:  
  - Conteúdo textual de cada página
  - Metadados do documento (título, autor, data)
  - Estrutura de páginas (quantidade, tamanho)
  - Fontes utilizadas
  - Estatísticas de objetos por tipo

3. **Transformações**:
  - Extração de todo texto para formato TXT;
  - Geração de sumário baseado na estrutura
  - Listagem hierárquica de objetos e suas dependências
  - Conversão de coordenadas de texto para formato mais legível

4. **Análise Avançada**:
  - Detecção de objetos não referenciados
  - Identificação de ciclos de referência
  - Análise de eficiência de armazenamento
  - Sugestões de otimização da estrutura

5. **O programa deve ler dois arquivos**:
  - O documento SimplePDF a ser analisado
  - Um arquivo de configuração que define parâmetros de análise e transformação

## Arquivo de Configuração

O arquivo sera disponibilizado com o mesmo nome do arquivo pdf, porem com o formato .py

Teste01.pdf
```
%SPDF-1.0
...
```

Teste01.py
```
extrair_texto=True
gerar_sumario=True
detectar_ciclos=True
nivel_detalhe='Completo'
validar_xref=True
```

Opções:
- **extrair_texto** (bool): Habilita a extração de texto do documento.
- **gerar_sumario** (bool): Gera um sumário do conteúdo do documento.
- **detectar_ciclos** (bool): Detecta possíveis referências cíclicas no documento.
- **nivel_detalhe** (str): Define o nível de detalhe da análise ('Completo', 'Basico', 'Nulo').
- **validar_xref** (bool): Valida as referências cruzadas dentro do documento.

## Saída do Programa

O programa deve gerar um relatório com as seguintes seções:
```
VALIDAÇÃO:
[OK ou ERRO] Estrutura geral
[OK ou ERRO] Sintaxe de objetos
[OK ou ERRO] Referências
[OK ou ERRO] Tabela xref
[Detalhes de erros, se houver]

ESTATÍSTICAS:
Total de objetos: X
Objetos por tipo: Catalog=1, Pages=1, Page=Y, Font=Z, ...
Total de páginas: Y
Tamanho do documento: W bytes
Overhead estrutural: V bytes (P%)

CONTEÚDO:
Título: [título do documento]
Autor: [autor do documento]
Data de criação: [data]
Texto extraído: [primeiros 200 caracteres...]

ÁRVORE DE OBJETOS:
1: Catalog
  +- 2: Pages
    +- 3: Page
    +- 4: Font
    +- 5: Contents (stream)
6: Metadata

ANÁLISE AVANÇADA:
[Resultados de análises específicas definidas no arquivo de configuração]
```

## AS TRANSFORMAÇÕES ESPECIFICADAS

1. **Extração de texto para formato TXT**

 Esta transformação envolve **localizar e extrair todo o conteúdo textual do documento SimplePDF**, que está disperso em diferentes objetos de conteúdo dentro de streams e **convertê-lo para um arquivo de texto simples e contínuo**.

 Por exemplo: No **Exemplo 2**, o texto está dentro de **objetos stream** com comandos como **(Relatório Financeiro: Primeiro Trimestre) Tj**. O programa precisa **identificar esses comandos, extrair o texto entre parênteses**, e **organizá-lo em uma sequência lógica**, **respeitando a ordem das páginas e a posição dos elementos de texto**.

2. **Geração de sumário baseado na estrutura**

  Esta transformação requer **analisar a estrutura hierárquica do documento (especialmente objetos do tipo Outlines)** e **gerar uma representação textual organizada que funcione como um sumário ou índice**.

  Por exemplo: No **Exemplo 2, os objetos 11, 12 e 13 formam uma estrutura de outlines com títulos** como "Resumo Executivo", "Detalhamento de Vendas" e "Projeções Futuras". **O programa precisa reconhecer essa hierarquia (quem é pai de quem, quem vem antes ou depois)** e **organizá-la em um formato de sumário**.

3. **Listagem hierárquica de objetos e suas dependências**

  Esta transformação envolve **mapear todas as relações entre objetos (quem referência quem)** e **apresentá-las em um formato visual hierárquico, como uma árvore**.

  Por exemplo: O **objeto raiz (Catalog)** referência **Pages**, que por sua vez referência **Page**, que referência **Font e Contents**. Isso forma uma **estrutura em árvore que precisa ser extraída e apresentada de forma clara**, como mostrado na seção ÁRVORE DE OBJETOS da saída esperada.

4. **Conversão de coordenadas de texto para formato mais legível**

  Esta transformação requer **interpretar as coordenadas e comandos de posicionamento de texto (como 100 700 Td)** e **convertê-los para um formato que seja mais intuitivo para humanos**, possivelmente relacionando-os com a posição na página.

  Por exemplo: Em vez de mostrar **100 700 Td**, o programa poderia **converter** isso **para** **"Posição: 100px da esquerda, 92px do topo da página" (considerando que em PDF as coordenadas Y geralmente começam da parte inferior da página)**.

## Exemplos

### Exemplo 1

Exemplo 1
```
%SPDF-1.0

1 0 obj
<< /Type /Catalog
/Pages 2 0 R
/Metadata 7 0 R
/Outlines 8 0 R
>>
endobj

2 0 obj
<< /Type /Pages
/Kids [3 0 R 4 0 R]
/Count 2
>>
endobj

3 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 6 0 R
>>
endobj

4 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 9 0 R
>>
endobj

5 0 obj
<< /Type /Font
/Subtype /Type1
/BaseFont /Helvetica
/FontDescriptor 10 0 R
>>
endobj

6 0 obj
<< /Length 175 >>
stream
BT
/F1 24 Tf
100 700 Td
(Relatório Financeiro: Primeiro Trimestre) Tj
/F1 12 Tf
0 -50 Td
(Total de vendas: 1423 unidades) Tj
0 -20 Td
(Receita total: 158432.75 reais) Tj
ET
endstream
endobj

7 0 obj
<< /Title (Relatório Financeiro Trimestral)
/Author (Departamento Financeiro)
/CreationDate (D:20230418090000)
/Keywords (finanças, relatório, trimestral)
/Version 1.2
/PageCount 2
>>
endobj

8 0 obj
<< /Type /Outlines
/Count 3
/First 11 0 R
/Last 13 0 R
>>
endobj

9 0 obj
<< /Length 322 >>
stream
BT
/F1 18 Tf
100 700 Td
(Detalhamento por Categoria) Tj
/F1 12 Tf
0 -40 Td
(Produto A: 523 unidades, valor unitário 89.99, total 47054.77) Tj
0 -20 Td
(Produto B: 347 unidades, valor unitário 125.50, total 43548.50) Tj
0 -20 Td
(Produto C: 553 unidades, valor unitário 122.65, total 67825.45) Tj
0 -40 Td
(Média de vendas por produto: 474.33 unidades) Tj
ET
endstream
endobj

10 0 obj
<< /Type /FontDescriptor
/FontName /Helvetica
/Flags 32
/FontBBox [-166 -225 1000 931]
/ItalicAngle 0
/Ascent 718
/Descent -207
/CapHeight 718
/StemV 88
/MissingWidth 278
>>
endobj

11 0 obj
<< /Title (Resumo Executivo)
/Parent 8 0 R
/Next 12 0 R
/Dest [3 0 R /FitH 792.0]
>>
endobj

12 0 obj
<< /Title (Detalhamento de Vendas)
/Parent 8 0 R
/Prev 11 0 R
/Next 13 0 R
/Dest [4 0 R /FitH 792.0]
>>
endobj

13 0 obj
<< /Title (Projeções Futuras)
/Parent 8 0 R
/Prev 12 0 R
/Dest [4 0 R /XYZ 100 450 1.25]
>>
endobj

14 0 obj
<< /Type /Metadata
/Subtype /XML
/Length 340
>>
stream
<?xpacket begin='' id='W5M0MpCehiHzreSzNTczkc9d'?>
<x:xmpmeta xmlns:x='adobe:ns:meta/'>
<rdf:RDF xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'>
<rdf:Description rdf:about=''
xmlns:pdf='http://ns.adobe.com/pdf/1.3/'>
<pdf:Producer>SimplePDF Generator 1.0</pdf:Producer>
</rdf:Description>
</rdf:RDF>
</x:xmpmeta>
<?xpacket end='w'?>
endstream
endobj
xref
0 15
0000000000 65535 f
0000000010 00000 n
0000000099 00000 n
0000000163 00000 n
0000000344 00000 n
0000000525 00000 n
0000000624 00000 n
0000000852 00000 n
0000001048 00000 n
0000001126 00000 n
0000001500 00000 n
0000001712 00000 n
0000001818 00000 n
0000001940 00000 n
0000002052 00000 n
trailer
<< /Size 15
/Root 1 0 R
/Info 7 0 R
>>
startxref
2500
%%EOF
```

### Exemplo 2

Exemplo 2
```
%SPDF-1.0

1 0 obj
<< /Type /Catalog
/Pages 2 0 R
/Metadata 7 0 R
/Outlines 8 0 R
>>
endobj

2 0 obj
<< /Type /Pages
/Kids [3 0 R 4 0 R]
/Count 2
>>
endobj

3 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 6 0 R
>>
endobj

4 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 9 0 R
>>
endobj

5 0 obj
<< /Type /Font
/Subtype /Type1
/BaseFont /Helvetica
/FontDescriptor 10 0 R
>>
endobj

6 0 obj
<< /Length 175 >>
stream
BT
/F1 24 Tf
100 700 Td
(Relatório Financeiro: Primeiro Trimestre) Tj
/F1 12 Tf
0 -50 Td
(Total de vendas: 1423 unidades) Tj
0 -20 Td
(Receita total: 158432.75 reais) Tj
ET
endstream
endobj

7 0 obj
<< /Title (Relatório Financeiro Trimestral)
/Author (Departamento Financeiro)
/CreationDate (D:20230418090000)
/Keywords (finanças, relatório, trimestral)
/Version 1.2
/PageCount 2
>>
endobj

8 0 obj
<< /Type /Outlines
/Count 3
/First 11 0 R
/Last 13 0 R
>>
endobj

9 0 obj
<< /Length 322 >>
stream
BT
/F1 18 Tf
100 700 Td
(Detalhamento por Categoria) Tj
/F1 12 Tf
0 -40 Td
(Produto A: 523 unidades, valor unitário 89.99, total 47054.77) Tj
0 -20 Td
(Produto B: 347 unidades, valor unitário 125.50, total 43548.50) Tj
0 -20 Td
(Produto C: 553 unidades, valor unitário 122.65, total 67825.45) Tj
0 -40 Td
(Média de vendas por produto: 474.33 unidades) Tj
ET
endstream
endobj

10 0 obj
<< /Type /FontDescriptor
/FontName /Helvetica
/Flags 32
/FontBBox [-166 -225 1000 931]
/ItalicAngle 0
/Ascent 718
/Descent -207
/CapHeight 718
/StemV 88
/MissingWidth 278
>>
endobj

11 0 obj
<< /Title (Resumo Executivo)
/Parent 8 0 R
/Next 12 0 R
/Dest [3 0 R /FitH 792.0]
>>
endobj

12 0 obj
<< /Title (Detalhamento de Vendas)
/Parent 8 0 R
/Prev 11 0 R
/Next 13 0 R
/Dest [4 0 R /FitH 792.0]
>>
endobj

13 0 obj
<< /Title (Projeções Futuras)
/Parent 8 0 R
/Prev 12 0 R
/Dest [4 0 R /XYZ 100 450 1.25]
>>
endobj

14 0 obj
<< /Type /Metadata
/Subtype /XML
/Length 340
>>
stream
<?xpacket begin='' id='W5M0MpCehiHzreSzNTczkc9d'?>
<x:xmpmeta xmlns:x='adobe:ns:meta/'>
<rdf:RDF xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'>
<rdf:Description rdf:about=''
xmlns:pdf='http://ns.adobe.com/pdf/1.3/'>
<pdf:Producer>SimplePDF Generator 1.0</pdf:Producer>
</rdf:Description>
</rdf:RDF>
</x:xmpmeta>
<?xpacket end='w'?>
endstream
endobj

xref
0 15
0000000000 65535 f
0000000010 00000 n
0000000099 00000 n
0000000163 00000 n
0000000344 00000 n
0000000525 00000 n
0000000624 00000 n
0000000852 00000 n
0000001048 00000 n
0000001126 00000 n
0000001500 00000 n
0000001712 00000 n
0000001818 00000 n
0000001940 00000 n
0000002052 00000 n

trailer
<< /Size 15
/Root 1 0 R
/Info 7 0 R
>>
startxref
2500

%%EOF
```

## Dicas

1. **Comece simples**: Crie primeiro documentos com poucos objetos e estrutura simples.
2. **Cálculo manual de offsets**: Para documentos simples, conte manualmente os bytes para cada
posição. Lembre-se de que cada caractere (incluindo espaços e quebras de linha) conta como
um byte.
3. **Uso de ferramentas**:  
  - Os alunos podem escrever um pequeno script auxiliar que conte o número de bytes
até cada objeto
  - Editores hexadecimais podem ser úteis para verificar posições
4. **Construção gradual**:  
  - Comece com o cabeçalho e objetos simples
  - Adicione um marcador temporário para a tabela xref (ex: "XREF_AQUI")
  - Conte a posição de cada objeto
  - Substitua o marcador pela tabela xref real
  - Adicione a seção trailer
5. **Formato de quebra de linha**: Certifique-se de usar consistentemente o mesmo formato de
quebra de linha (CRLF ou LF) ao contar bytes.
6. **Verificação do documento**:  
Após criar o documento, percorra-o byte a byte para verificar se os offsets na tabela xref apontam corretamente para o início de cada objeto

**CASOS DE TESTE IMPORTANTES**

Os alunos devem criar documentos de teste que cubram:
1. **Estrutura básica**: Documento mínimo com catálogo e páginas
2. **Reais vs. Inteiros**: Uso de números inteiros e de ponto flutuante
3. **Tipos de dados**: Todas as variações (strings, nomes, arrays, dicionários)
4. **Referências circulares**: Objetos que se referenciam mutuamente
5. **Conteúdo de texto**: Objetos stream com comandos de texto
6. **Estrutura hierárquica complexa**: Vários níveis de referência entre objetos
7. **Metadados completos**: Informações detalhadas no objeto de metadados
Criando arquivos de teste apropriados, os alunos poderão verificar se suas implementações lidam
corretamente com todos os aspectos do formato SimplePDF, especialmente a interpretação da
tabela xref, que é crucial para a navegação eficiente dentro do documento.

# Código separado por celulas (Google Colab Style)

**Downloading pdfs and configs (at least 3 examples)**

In [2]:
!wget -nc https://raw.githubusercontent.com/JoaoVLima/ActualSimplePDFValidator/main/exemplos/exemplo3.pdf
!wget -nc https://raw.githubusercontent.com/JoaoVLima/ActualSimplePDFValidator/main/exemplos/exemplosimples.py

--2025-03-27 18:02:18--  https://raw.githubusercontent.com/JoaoVLima/ActualSimplePDFValidator/main/exemplos/exemplo3.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 604 [text/plain]
Saving to: ‘exemplo3.pdf’

exemplo3.pdf        100%[===================>]     604  --.-KB/s    in 0s      

2025-03-27 18:02:18 (20.5 MB/s) - ‘exemplo3.pdf’ saved [604/604]

--2025-03-27 18:02:18--  https://raw.githubusercontent.com/JoaoVLima/ActualSimplePDFValidator/main/exemplos/exemplosimples.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56

**Config**

In [3]:
extrair_texto=None
gerar_sumario=None
detectar_ciclos=None
nivel_detalhe=None
validar_xref=None

In [4]:
from exemplosimples import *

In [5]:
print(extrair_texto)
print(gerar_sumario)
print(detectar_ciclos)
print(nivel_detalhe)
print(validar_xref)

True
True
True
Completo
True


**Read File**

In [6]:
def read_pdf(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return text

In [7]:
FILE = read_pdf('exemplo3.pdf')

In [8]:
FILE

'%SPDF-1.0\n1 0 obj\n<<\n/Type /Catalog\n/Pages 3 0 R\n/Outlines 2 0 R\n>>\nendobj\n2 0 obj\n<<\n/Type /Outlines\n/Count 0\n>>\nendobj\n3 0 obj\n<<\n/Type /Pages\n/Count 1\n/Kids [ 4 0 R ]\n>>\nendobj\n4 0 obj\n<<\n/Type /Page\n/Parent 3 0 R\n/Resources << /ProcSet 6 0 R >>\n/MediaBox [ 0 0 612 792 ]\n/Contents 5 0 R\n>>\nendobj\n5 0 obj\n<<\n/Length 35\n>>\nstream\n%place page marking operators here\nendstream\nendobj\n6 0 obj\n[ /PDF ]\nendobj\nxref\n0 7\n0000000000 65535 f\n0000000010 00000 n\n0000000075 00000 n\n0000000121 00000 n\n0000000180 00000 n\n0000000301 00000 n\n0000000385 00000 n\ntrailer\n<<\n/Size 7\n/Root 1 0 R\n>>\nstartxref\n409\n%%EOF'

**Getting info and Checking them**

In [9]:
FILE_SIZE = len(FILE)
FILE_SIZE

604

In [10]:
def check_header(file):
    return file.startswith('%SPDF-') or file.startswith('%PDF-')

In [11]:
check_header(FILE)

True

In [12]:
def get_startbody(file):
    for i in range(256):
        if file[i] == '\n': # get the second line
            return i+1

In [13]:
STARTBODY = get_startbody(FILE)
STARTBODY

10

In [14]:
def check_eof(file):
    return file.endswith('%%EOF')

In [15]:
check_eof(FILE)

True

In [16]:
def get_startxref(file, filesize):
    end = filesize-6 # ignoring this '\n%%EOF'
    for i in range(end-1,0,-1):
        if file[i] == '\n': # get only one line
            return int(file[i+1:end])

In [17]:
STARTXREF = get_startxref(FILE, FILE_SIZE)
STARTXREF

409

In [18]:
BODY = FILE[STARTBODY:STARTXREF]
BODY

'1 0 obj\n<<\n/Type /Catalog\n/Pages 3 0 R\n/Outlines 2 0 R\n>>\nendobj\n2 0 obj\n<<\n/Type /Outlines\n/Count 0\n>>\nendobj\n3 0 obj\n<<\n/Type /Pages\n/Count 1\n/Kids [ 4 0 R ]\n>>\nendobj\n4 0 obj\n<<\n/Type /Page\n/Parent 3 0 R\n/Resources << /ProcSet 6 0 R >>\n/MediaBox [ 0 0 612 792 ]\n/Contents 5 0 R\n>>\nendobj\n5 0 obj\n<<\n/Length 35\n>>\nstream\n%place page marking operators here\nendstream\nendobj\n6 0 obj\n[ /PDF ]\nendobj\n'

In [19]:
BODY_SIZE = len(BODY)
BODY_SIZE

399

In [20]:
STARTXREF_KEYWORD_OFFSET = FILE_SIZE-6-len(str(STARTXREF))-10 # f'startxref\n{xref_address}\n%%EOF'

In [21]:
def get_starttrailer(file, startxref, startxref_keyword_offset):
    end = startxref_keyword_offset # ignoring this f'startxref\n{xref_address}\n%%EOF'
    for i in range(startxref,end):
        if file[i] == 't': # trailler
            return i

In [22]:
STARTTRAILER = get_starttrailer(FILE, STARTXREF, STARTXREF_KEYWORD_OFFSET)
STARTTRAILER

551

In [23]:
XREF = FILE[STARTXREF:STARTTRAILER]
XREF

'xref\n0 7\n0000000000 65535 f\n0000000010 00000 n\n0000000075 00000 n\n0000000121 00000 n\n0000000180 00000 n\n0000000301 00000 n\n0000000385 00000 n\n'

In [24]:
# Trailler is  'trailer << <trailer key–value pair>+ >> startxref <cross-reference table start address> %%EOF'
# but i am only considering 'trailer << <trailer key–value pair>+ >> '
TRAILER = FILE[STARTTRAILER:STARTXREF_KEYWORD_OFFSET]
TRAILER

'trailer\n<<\n/Size 7\n/Root 1 0 R\n>>\n'

**Converting dictionary from pdf to dictionary in python**

In [25]:
def string_to_dict(dict_str):
    tokens = dict_str.split()
    return tokens_to_dict(tokens)


def tokens_to_dict(tokens, index=-1, index_return=False):
    data = {}
    key = None
    value = []
    i = index
    while i < len(tokens):
        i+=1
        token = tokens[i]

        if token == '<<':
            if key:
                data[key], i = tokens_to_dict(tokens, i, True)
            key = None
            value = []
        elif token == '>>' or token == 'endobj':
            if len(value) == 1:
                value = value[0]  # ['7'] => '7'
            if key:
                data[key] = value
            break
        elif token.startswith('/'):
            if key:
                if value:
                    if len(value) == 1:
                        value = value[0] # ['7'] => '7'
                    data[key] = value
                    key = None
                    value = []
                else:
                    value = token
                    data[key] = value # '/Type': '/Pages'
                    key = None
                    value = []
                    continue
            key = token
            value = []
        else:
            if not key:
                continue
            value.append(token)

    if index_return:
        return data, i
    return data

In [26]:
TRAILER_DICT = string_to_dict(TRAILER)
TRAILER_DICT

{'/Size': '7', '/Root': ['1', '0', 'R']}

**Checking required keys on trailer and getting info from it**

In [27]:
def check_trailer(trailer_dict):
    required_tags = ['/Size','/Root']
    # tags = ['/Info','/Prev']
    for tag in required_tags:
        if tag not in trailer_dict:
            return False

    return True

In [28]:
check_trailer(TRAILER_DICT)

True

In [29]:
TRAILER_SIZE = int(TRAILER_DICT['/Size'])
TRAILER_SIZE

7

In [30]:
TRAILER_ROOT = TRAILER_DICT['/Root']
TRAILER_ROOT

['1', '0', 'R']

**Converting xref into a list of sections(list)**

In [31]:
def xref_to_list(xref):
    lines = xref.split('\n')
    lines = [line.split() for line in lines if line]

    first = int(lines[1][0])
    qtd = int(lines[1][1])

    lines = lines[2:]  # removing 'xref' and '0 7', 2 first lines

    sections = []
    objects = [[first,qtd]]

    for line in lines:
        if len(line) == 2:
            sections.append(objects)
            first = int(line[0])
            qtd = int(line[1])
            objects = [[first,qtd]]
            continue

        line[0] = int(line[0])
        line[1] = int(line[1])
        objects.append(line)

    if objects:
        sections.append(objects)

    return sections

In [32]:
XREF_LIST = xref_to_list(XREF)
XREF_LIST

[[[0, 7],
  [0, 65535, 'f'],
  [10, 0, 'n'],
  [75, 0, 'n'],
  [121, 0, 'n'],
  [180, 0, 'n'],
  [301, 0, 'n'],
  [385, 0, 'n']]]

In [33]:
def check_xref_size(xref_list, trailer_size):
    size = 0
    for section in xref_list:
        info = section[0]
        first = info[0]
        qtd = info[1]
        section_size = len(section[1:])

        if qtd != section_size:
            return False

        size += section_size

    if size != trailer_size:
        return False

    return True

In [34]:
check_xref_size(XREF_LIST, TRAILER_SIZE)

True

In [35]:
# f = object_id gen_num f
# n = byte_offset gen_num n

def get_xref_addresses(xref_list):
    xref_addresses = {}
    for section in xref_list:
        info = section[0]
        first = info[0]
        qtd = info[1]

        section = section[1:] # ignoring info, first element

        for i in range(qtd):
            obj = section[i]

            if obj[2] == 'f':
                continue # ignoring validation on 'f'

            byte_offset = obj[0]

            xref_addresses[first+i] = byte_offset

    return xref_addresses

In [36]:
XREF_ADDRESSES = get_xref_addresses(XREF_LIST)
XREF_ADDRESSES

{1: 10, 2: 75, 3: 121, 4: 180, 5: 301, 6: 385}

In [37]:
def check_xref_addresses(xref_addresses, body, start_body=10): # start_body=10 for spdf header
    for obj_id, address in xref_addresses.items():

        if body[address-start_body] != str(obj_id):
            return False

    return True

In [38]:
check_xref_addresses(XREF_ADDRESSES, BODY, STARTBODY)

True

**Creating OBJECTS dictionary, putting the addresses(byte_offsets-start_body) in it**

In [39]:
OBJECTS = {1:{}}

In [40]:
def update_xref_addresses(objects, xref_addresses, start_body=10): # start_body=10 for spdf header)
    for obj_id, address in xref_addresses.items():
        if obj_id not in objects:
            objects[obj_id] = {}
        objects[obj_id]['address'] = address - start_body

In [41]:
update_xref_addresses(OBJECTS, XREF_ADDRESSES)
OBJECTS

{1: {'address': 0},
 2: {'address': 65},
 3: {'address': 111},
 4: {'address': 170},
 5: {'address': 291},
 6: {'address': 375}}

**Putting content from objects inside OBJECTS dictionary**

In [42]:
def update_contents(objects, body, body_size):
    for obj_id, obj in objects.items():
        address = obj['address']
        i = address

        while(i<body_size):
            if body[i:i+6] == 'endobj':
                break
            i+=1

        obj['content'] = body[address:i+6]

In [43]:
update_contents(OBJECTS, BODY, BODY_SIZE)
OBJECTS

{1: {'address': 0,
  'content': '1 0 obj\n<<\n/Type /Catalog\n/Pages 3 0 R\n/Outlines 2 0 R\n>>\nendobj'},
 2: {'address': 65,
  'content': '2 0 obj\n<<\n/Type /Outlines\n/Count 0\n>>\nendobj'},
 3: {'address': 111,
  'content': '3 0 obj\n<<\n/Type /Pages\n/Count 1\n/Kids [ 4 0 R ]\n>>\nendobj'},
 4: {'address': 170,
  'content': '4 0 obj\n<<\n/Type /Page\n/Parent 3 0 R\n/Resources << /ProcSet 6 0 R >>\n/MediaBox [ 0 0 612 792 ]\n/Contents 5 0 R\n>>\nendobj'},
 5: {'address': 291,
  'content': '5 0 obj\n<<\n/Length 35\n>>\nstream\n%place page marking operators here\nendstream\nendobj'},
 6: {'address': 375, 'content': '6 0 obj\n[ /PDF ]\nendobj'}}

In [44]:
def remove_comments(objects):
    for obj_id, obj in objects.items():
        content = obj['content']
        new_content = ''
        start_comment = False

        for i in range(len(content)):
            if start_comment:
                if content[i] == '\n':
                    new_content += '\n'
                    start_comment = None
                    continue
            elif content[i] == '%':
                start_comment = True
                continue
            else:
                new_content += content[i]

        obj['content'] = new_content

In [45]:
remove_comments(OBJECTS)
OBJECTS

{1: {'address': 0,
  'content': '1 0 obj\n<<\n/Type /Catalog\n/Pages 3 0 R\n/Outlines 2 0 R\n>>\nendobj'},
 2: {'address': 65,
  'content': '2 0 obj\n<<\n/Type /Outlines\n/Count 0\n>>\nendobj'},
 3: {'address': 111,
  'content': '3 0 obj\n<<\n/Type /Pages\n/Count 1\n/Kids [ 4 0 R ]\n>>\nendobj'},
 4: {'address': 170,
  'content': '4 0 obj\n<<\n/Type /Page\n/Parent 3 0 R\n/Resources << /ProcSet 6 0 R >>\n/MediaBox [ 0 0 612 792 ]\n/Contents 5 0 R\n>>\nendobj'},
 5: {'address': 291,
  'content': '5 0 obj\n<<\n/Length 35\n>>\nstream\n\nendstream\nendobj'},
 6: {'address': 375, 'content': '6 0 obj\n[ /PDF ]\nendobj'}}

In [47]:
def check_references(objects):
    for obj in objects.values():
        content = obj['content']
        tokens = content.split()

        refs = get_reference_id(tokens)
        for id in refs:
            if id not in objects:
                return False

    return True


In [48]:
check_references(OBJECTS)

True

In [49]:
def get_dictionaries(objects):
    for obj in objects.values():
        obj['dictionary'] = string_to_dict(obj['content'])

In [50]:
get_dictionaries(OBJECTS)
OBJECTS

{1: {'address': 0,
  'content': '1 0 obj\n<<\n/Type /Catalog\n/Pages 3 0 R\n/Outlines 2 0 R\n>>\nendobj',
  'dictionary': {'/Type': '/Catalog',
   '/Pages': ['3', '0', 'R'],
   '/Outlines': ['2', '0', 'R']}},
 2: {'address': 65,
  'content': '2 0 obj\n<<\n/Type /Outlines\n/Count 0\n>>\nendobj',
  'dictionary': {'/Type': '/Outlines', '/Count': '0'}},
 3: {'address': 111,
  'content': '3 0 obj\n<<\n/Type /Pages\n/Count 1\n/Kids [ 4 0 R ]\n>>\nendobj',
  'dictionary': {'/Type': '/Pages',
   '/Count': '1',
   '/Kids': ['[', '4', '0', 'R', ']']}},
 4: {'address': 170,
  'content': '4 0 obj\n<<\n/Type /Page\n/Parent 3 0 R\n/Resources << /ProcSet 6 0 R >>\n/MediaBox [ 0 0 612 792 ]\n/Contents 5 0 R\n>>\nendobj',
  'dictionary': {'/Type': '/Page',
   '/Parent': ['3', '0', 'R'],
   '/Resources': {'/ProcSet': ['6', '0', 'R']},
   '/MediaBox': ['[', '0', '0', '612', '792', ']'],
   '/Contents': ['5', '0', 'R']}},
 5: {'address': 291,
  'content': '5 0 obj\n<<\n/Length 35\n>>\nstream\n\nendstream\

In [51]:
class Node:
    def __init__(self, info=''):
        self.info = info
        self.children = []

    def __str__(self):
        return self.info

    def add_child(self, child):
        self.children.append(child)
        return True

    def height(self):
        if not self.children:
            return 1
        return 1 + max([child.height() for child in self.children])

    def find_node(self, target):
        if self.info == target:
            return self

        for child in self.children:
            if child.info == target:
                return child
            found = child.find_node(target)
            if found:
                return found

    def print(self, level=0):
        result = "\t" * level + str(self) + "\n"
        for child in self.children:
            result += child.print(level + 1)
        return result


class Tree:
    def __init__(self, root=None):
        if root and not isinstance(root, Node):
            root = Node(root)
        self.root = root

    def __str__(self):
        return self.print()

    def is_empty(self):
        return self.root is None

    def height(self):
        return self.root.height()

    def find_node(self, target):
        if self.is_empty():
            return None
        return self.root.find_node(target)

    def add_child(self, target, node):
        if self.is_empty():
            return None

        if not isinstance(target, Node):
            target = self.find_node(target)

        if not isinstance(node, Node):
            node = Node(node)

        return target.add_child(node)

    def print(self):
        if self.is_empty():
            return ''
        return self.root.print()


In [52]:
def get_reference_id(tokens):
    ids = []
    for i in range(len(tokens)):
        token = tokens[i]
        if token == 'R':
            ref_id = tokens[i - 2]
            ref_id = int(ref_id)
            ids.append(ref_id)

    return ids

In [82]:
def references_recursive(object):
    references = []
    if 'dictionary' in object:
        object = object['dictionary']
    for key, value in object.items():
        if key == '/Parent':
            continue
        if isinstance(value, dict):
            references += references_recursive(value)
            continue
        references += get_reference_id(value)

    return references

In [83]:
def get_tree(objects, root):
    root_object = objects[root]
    root = f"{root}: {root_object['dictionary'].get('/Type', 'obj')}"
    tree = Tree(root)

    references = references_recursive(root_object)

    for ref_id in references:
        node = get_tree(objects, ref_id).root
        tree.add_child(root, node)

    return tree

In [84]:
tree = get_tree(OBJECTS, get_reference_id(TRAILER_ROOT)[0])

In [85]:
print(tree)

1: /Catalog
	3: /Pages
		4: /Page
			6: obj
			5: obj
	2: /Outlines

